In [ ]:
!# Instalação correta das dependências
!pip install -U google-generativeai gradio gTTS openai-whisper python-dotenv

In [ ]:
import os
import whisper
import gradio as gr
import google.generativeai as genai
from google.api_core import exceptions as google_exceptions
from gtts import gTTS
import uuid
import logging
import getpass
from dotenv import load_dotenv

# Isso impede que o Python tente sair pelo proxy para falar com o localhost
os.environ['NO_PROXY'] = 'localhost,127.0.0.1,0.0.0.0'


In [ ]:
# --- Configuração Inicial ---
load_dotenv()
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Autenticação
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    # Fallback para input manual se não estiver no .env
    api_key = getpass.getpass("Digite sua Google API Key: ")

genai.configure(api_key=api_key)

# --- LÓGICA DE SELEÇÃO DINÂMICA DE MODELO ---
def obter_modelo_dinamico():
    """
    Lista os modelos disponíveis na sua conta e seleciona o melhor candidato
    automaticamente, evitando erros de 404 por nomes hardcoded.
    """
    print("🔍 Consultando API do Google para listar modelos disponíveis...")

    try:
        # Pega todos os modelos que suportam gerar texto ('generateContent')
        todos_modelos = [m for m in genai.list_models() if 'generateContent' in m.supported_generation_methods]

        # Ordena preferência: Flash (rápido) > Pro (inteligente) > Qualquer outro Gemini
        # Isso cria uma lista de prioridade baseada no nome
        def pontuacao_modelo(m):
            nome = m.name.lower()
            if '1.5-flash' in nome: return 3
            if '1.5-pro' in nome: return 2
            if 'gemini' in nome: return 1
            return 0

        # Ordena a lista pela pontuação (do maior para o menor)
        todos_modelos.sort(key=pontuacao_modelo, reverse=True)

        if not todos_modelos:
            raise ValueError("Nenhum modelo compatível encontrado na sua conta Google AI.")

        # Pega o campeão
        modelo_escolhido = todos_modelos[0]
        nome_real = modelo_escolhido.name # Ex: 'models/gemini-1.5-flash-001'

        print(f"✅ Modelo selecionado automaticamente: {nome_real}")
        print(f"   (Descrição: {modelo_escolhido.description})")

        return genai.GenerativeModel(nome_real)

    except Exception as e:
        logging.critical(f"Falha ao listar modelos. Erro: {e}")
        # Último recurso desesperado: tenta o legacy puro se a listagem falhar
        return genai.GenerativeModel('gemini-pro')

# Inicializa
whisper_model = whisper.load_model("base")
gemini_model = obter_modelo_dinamico() # <--- Aqui a mágica acontece

# --- Processamento (O resto segue igual) ---
def processar_audio(audio_path):
    if audio_path is None:
        return "Nenhum áudio.", "Sem input.", None

    request_id = str(uuid.uuid4())[:8]

    # 1. Transcrição
    try:
        audio = whisper_model.transcribe(audio_path, fp16=False)
        texto = audio["text"]
    except Exception as e:
        return f"Erro Whisper: {e}", "", None

    # 2. LLM
    try:
        logging.info(f"[{request_id}] Gerando resposta...")
        response = gemini_model.generate_content(texto)
        resposta = response.text
    except Exception as e:
        logging.error(f"Erro Gemini: {e}")
        resposta = f"Erro na API: {str(e)}"

    # 3. TTS
    path_audio = None
    try:
        if resposta and not "Erro" in resposta:
            clean_text = resposta.replace("*", "")
            tts = gTTS(text=clean_text, lang='pt', slow=False)
            path_audio = f"resp_{request_id}.mp3"
            tts.save(path_audio)
    except Exception:
        pass

    return texto, resposta, path_audio

# Interface
iface = gr.Interface(
    fn=processar_audio,
    inputs=gr.Audio(sources=["microphone"], type="filepath"),
    outputs=[gr.Textbox(label="Você"), gr.Textbox(label="IA"), "audio"],
    title="Assistente Gemini (Auto-Discovery)"
)

if __name__ == "__main__":
  # server_name="127.0.0.1" força rodar localmente sem tentar abrir para rede
  iface.launch(debug=True, server_name="127.0.0.1", server_port=7860, share=False)
